#### Import Dependencies

In [18]:
import os
import time
import json
from datetime import datetime
import requests
from dotenv import load_dotenv
from pyspark.sql import SparkSession

#### Fetch Movies Data 

In [ ]:
def fetch_movie_data_spark(movie_ids, save_path="data/raw"):
    # Load env variables
    load_dotenv()
    api_key = os.getenv('API_KEY')
    base_url = os.getenv('BASE_url') 

    # Initialize Spark
    spark = SparkSession.builder.appName("FetchTMDBMovies").getOrCreate()

    movies_data = []
    for movie_id in movie_ids:
        for attempt in range(3):
            try:
                url = f'{base_url}{movie_id}?api_key={api_key}&append_to_response=credits'
                r = requests.get(url)
                if r.status_code == 200:
                    movies_data.append(r.json())
                    break
                elif r.status_code == 429:
                    print("Rate limit hit. Waiting 3 seconds...")
                    time.sleep(3)
                else:
                    print(f"Failed: {movie_id} (Status {r.status_code})")
                    time.sleep(1)
            except Exception as e:
                print(f"Error: {e}")
                time.sleep(1)

        time.sleep(0.3)  # ~3 req/sec max

    # Convert to RDD and DataFrame
    rdd = spark.sparkContext.parallelize(movies_data)
    df = spark.read.json(rdd)

    # Save using timestamp
    timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
    file_name = f"moviesData_{timestamp}.json"
    output_path = os.path.join(save_path, file_name)
    os.makedirs(save_path, exist_ok=True)

    #Write using native Python (bypasses Spark/Hadoop)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(movies_data, f, ensure_ascii=False, indent=2)

    print(f"Data saved to: {output_path}")
    return df


In [17]:
movie_ids = [
    0, 299534, 19995, 140607, 299536, 597, 135397, 420818, 24428,
    168259, 99861, 284054, 12445, 181808, 330457, 351286, 109445,
    321612, 260513
]

df = fetch_movie_data_spark(movie_ids)
df.show(truncate=False)

Failed: 0 (Status 404)
Failed: 0 (Status 404)
Failed: 0 (Status 404)
✅ Data saved to: data/raw\moviesData_23-04-2025_15-49-52.json


Py4JJavaError: An error occurred while calling o105.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2385)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2414)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2433)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:530)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:483)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:61)
	at org.apache.spark.sql.Dataset.collectFromPlan(Dataset.scala:4333)
	at org.apache.spark.sql.Dataset.$anonfun$head$1(Dataset.scala:3316)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$2(Dataset.scala:4323)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:546)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:4321)
	at org.apache.spark.sql.execution.SQL